In [6]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [7]:
# %cd '/content/gdrive/My Drive/CARD CHURN/'

In [8]:
import json
import pandas as pd
from s1_data_fetch import s1_fetch_data_from_sql
from s2_data_cleaning import s2_clean_and_prepare_data
from s3_model_utils import s3_load_model, s3_make_predictions
from s4_file_utils import s4_save_predictions

In [9]:
def load_config(config_file='config.txt'):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

In [10]:
def run_prediction_pipeline(config):
    user_sql = config.get('user_sql')
    pass_sql = config.get('pass_sql')
    list_month = config.get('list_month')
    sample_test_oot_path = config.get('sample_test_oot_path')
    model_path = config.get('model_path')
    threshold = config.get('threshold', 0.7)
    output_path = config.get('output_path')
    
    # Step 1: Fetch data from SQL for the specified list_month
    data = s1_fetch_data_from_sql(user_sql=user_sql, pass_sql=pass_sql, list_month=list_month, sample_test_oot_path=sample_test_oot_path)

    # Step 2: Clean the data
    cleaned_data = s2_clean_and_prepare_data(sample_test_oot_path)
    
    # Step 3: Load the trained model
    model = s3_load_model(model_path)
    
    # Step 4: Make predictions using the model with the specified threshold
    predictions = s3_make_predictions(model=model, cleaned_data=cleaned_data, threshold=threshold)
    print(predictions.shape)
    
    # Step 5: Save predictions to a CSV file
    s4_save_predictions(predictions, output_path)

In [11]:
if __name__ == "__main__":    
    config = load_config()
    run_prediction_pipeline(config)


    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_TYPE = 'BASE TABLE'
    AND (TABLE_NAME LIKE 'TMP%')
    AND (TABLE_NAME LIKE '%CARD%')
    
         TABLE_NAME
0  TMP1_RF_CARD_FCT
1  TMP2_RF_CARD_FCT
202408


        SELECT CUSTOMER_ID, MONTH_ID_N as MONTH_ID, Y FROM CHURN_DEFINE_CHURN_CREDIT_LEADTIME_SAMPLE
        WHERE MONTH_ID_N = 202408
        AND Y=0
        
TMP1_RF_CARD_FCT


            SELECT * FROM TMP1_RF_CARD_FCT WHERE MONTH_ID = 202408
            
TMP2_RF_CARD_FCT


            SELECT * FROM TMP2_RF_CARD_FCT WHERE MONTH_ID = 202408
            


(51190, 5)
Predictions saved to sample_test_oot_folder/df_predict.csv
